In [ ]:
#Imports para executar o treinamento, em casos de duvidas fico a disposicao pois foi um caos ajustar todas as dependencias

# !pip install tensorflow==2.12.0 gym==0.21.0 keras-rl2=1.0.5 gym[atari] gymnasium[accept-rom-license] gymnasium[atari]


# Testando os imports

In [11]:
import gym
import random

In [12]:
from colorama import just_fix_windows_console, init, Back, Fore

init(autoreset=True)

# use Colorama to make Termcolor work on Windows too
just_fix_windows_console()
environment_name = "ALE/SpaceInvaders-v5"

In [13]:
env = gym.make(environment_name)
height, width, channels = env.observation_space.shape
actions = env.action_space.n

In [14]:
env.unwrapped.get_action_meanings()

['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

# Criando e treinando o modelo

In [15]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam
import tensorflow as tf


In [16]:
def build_model(height, width, channels, actions):
    model = Sequential()
    model.add(Convolution2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3,height, width, channels)))
    model.add(Convolution2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Convolution2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [18]:
model = build_model(height, width, channels, actions)

del model

model = build_model(height, width, channels, actions)

In [19]:
from rl.agents import DQNAgent
from rl.memory import SequentialMemory
from rl.policy import LinearAnnealedPolicy, EpsGreedyQPolicy
import tensorflow as tf

In [20]:
def build_agent(model, actions):
    policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000)
    memory = SequentialMemory(limit=10000, window_length=3)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                  enable_dueling_network=True, dueling_type='avg', 
                   nb_actions=actions, nb_steps_warmup=1000
                  )
    return dqn

In [21]:
opt = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001)

dqn = build_agent(model, actions)
dqn.compile(optimizer=opt)

In [22]:
dqn.fit(env, nb_steps=100000, visualize=False, verbose=2)

Training for 100000 steps ...


C:\Users\Eiji\AppData\Roaming\Python\Python310\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


   296/100000: episode: 1, duration: 6.131s, episode steps: 296, steps per second:  48, episode reward: 50.000, mean reward:  0.169 [ 0.000, 20.000], mean action: 2.304 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --
   868/100000: episode: 2, duration: 11.455s, episode steps: 572, steps per second:  50, episode reward: 135.000, mean reward:  0.236 [ 0.000, 30.000], mean action: 2.302 [0.000, 5.000],  loss: --, mean_q: --, mean_eps: --


C:\Users\Eiji\AppData\Roaming\Python\Python310\site-packages\keras\engine\training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


  1424/100000: episode: 3, duration: 321.734s, episode steps: 556, steps per second:   2, episode reward: 170.000, mean reward:  0.306 [ 0.000, 30.000], mean action: 2.674 [0.000, 5.000],  loss: 8.936710, mean_q: 7.363911, mean_eps: 0.890920
  2138/100000: episode: 4, duration: 551.843s, episode steps: 714, steps per second:   1, episode reward: 225.000, mean reward:  0.315 [ 0.000, 30.000], mean action: 2.419 [0.000, 5.000],  loss: 1.546925, mean_q: 7.022682, mean_eps: 0.839755
  3126/100000: episode: 5, duration: 774.366s, episode steps: 988, steps per second:   1, episode reward: 290.000, mean reward:  0.294 [ 0.000, 30.000], mean action: 2.503 [0.000, 5.000],  loss: 0.914421, mean_q: 6.810249, mean_eps: 0.763165
  3532/100000: episode: 6, duration: 382.236s, episode steps: 406, steps per second:   1, episode reward: 155.000, mean reward:  0.382 [ 0.000, 30.000], mean action: 2.448 [0.000, 5.000],  loss: 0.662070, mean_q: 6.732343, mean_eps: 0.700435
  3971/100000: episode: 7, durat

 17673/100000: episode: 36, duration: 1116.639s, episode steps: 603, steps per second:   1, episode reward: 140.000, mean reward:  0.232 [ 0.000, 25.000], mean action: 2.670 [0.000, 5.000],  loss: 0.212034, mean_q: 6.174518, mean_eps: 0.100000
 18500/100000: episode: 37, duration: 1527.724s, episode steps: 827, steps per second:   1, episode reward: 155.000, mean reward:  0.187 [ 0.000, 30.000], mean action: 2.064 [0.000, 5.000],  loss: 0.271975, mean_q: 6.156345, mean_eps: 0.100000
 18993/100000: episode: 38, duration: 911.348s, episode steps: 493, steps per second:   1, episode reward: 100.000, mean reward:  0.203 [ 0.000, 30.000], mean action: 1.935 [0.000, 5.000],  loss: 0.180735, mean_q: 6.173980, mean_eps: 0.100000
 19348/100000: episode: 39, duration: 657.619s, episode steps: 355, steps per second:   1, episode reward: 40.000, mean reward:  0.113 [ 0.000, 15.000], mean action: 2.865 [0.000, 5.000],  loss: 0.394117, mean_q: 6.186566, mean_eps: 0.100000
 19822/100000: episode: 40,

 34923/100000: episode: 69, duration: 626.560s, episode steps: 365, steps per second:   1, episode reward: 60.000, mean reward:  0.164 [ 0.000, 25.000], mean action: 2.403 [0.000, 5.000],  loss: 2.026346, mean_q: 7.070735, mean_eps: 0.100000
 35784/100000: episode: 70, duration: 1473.617s, episode steps: 861, steps per second:   1, episode reward: 490.000, mean reward:  0.569 [ 0.000, 200.000], mean action: 2.027 [0.000, 5.000],  loss: 3.022921, mean_q: 7.065766, mean_eps: 0.100000
done, took 58860.429 seconds


# Testando o modelo

In [25]:
scores = dqn.test(env, nb_episodes=10, visualize=True)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: 70.000, steps: 469
Episode 2: reward: 55.000, steps: 438
Episode 3: reward: 110.000, steps: 497
Episode 4: reward: 140.000, steps: 677
Episode 5: reward: 30.000, steps: 430
Episode 6: reward: 80.000, steps: 303
Episode 7: reward: 75.000, steps: 407
Episode 8: reward: 145.000, steps: 498
Episode 9: reward: 640.000, steps: 1100
Episode 10: reward: 150.000, steps: 553
149.5


In [23]:
dqn.save_weights('dqn_steps_35784_mr_0569.h5f')

In [22]:
del model, dqn

In [32]:
dqn.load_weights('dqn_114323.h5f')